In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

# 다중공선성 진단용 VIF 계산 함수 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

### 데이터 병합

In [2]:
# 채널정보 파일 읽기
channel_df = pd.read_parquet('open/concat/2018_잔액정보.parquet')

# 회원정보 파일 읽기
member_df  = pd.read_parquet('open/concat/2018_회원정보.parquet')

# 회원정보에서 ID와 Segment 컬럼만 추출
member_seg = member_df[['ID', 'Segment']]

# 중복된 ID 개수 확인
dup_count = member_seg['ID'].duplicated().sum()
print(f'중복된 ID 개수: {dup_count}')

# 중복된 ID를 첫 번째 항목만 남기고 제거
member_seg_unique = member_seg.drop_duplicates(subset='ID', keep='first')
print(f'중복 제거 후 행 수: {len(member_seg_unique)}')

# 채널정보에 Segment 컬럼 병합 (1:1 조인 보장)
df = channel_df.merge(member_seg_unique, on='ID', how='left')

# 결과 확인
print("병합 후 데이터프레임 크기:", df.shape)
print(df.head())

# 병합된 파일 저장
df.to_parquet('2018_잔액정보_with_segment.parquet', index=False)

중복된 ID 개수: 2500000
중복 제거 후 행 수: 500000
병합 후 데이터프레임 크기: (3000000, 83)
     기준년월            ID  잔액_일시불_B0M  잔액_할부_B0M  잔액_현금서비스_B0M  잔액_리볼빙일시불이월_B0M  \
0  201807  TRAIN_000000         998        962         22971                0   
1  201807  TRAIN_000001        2565       2390             0                0   
2  201807  TRAIN_000002        5312       5113         21531             6795   
3  201807  TRAIN_000003         730       5025         26284                0   
4  201807  TRAIN_000004           0          0             0                0   

   잔액_리볼빙CA이월_B0M  잔액_카드론_B0M  월중평잔_일시불_B0M  월중평잔_할부_B0M  ...  평잔_일시불_6M  \
0               0           0          1084          547  ...       2440   
1               0           0          4090         2553  ...       2677   
2               0           0          5006         8778  ...       9118   
3               0           0           487         5607  ...        884   
4               0           0             0            0  ...   

In [3]:
# 데이터를 불러온다.
df = pd.read_parquet('2018_잔액정보_with_segment.parquet')
df

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M,Segment
0,201807,TRAIN_000000,998,962,22971,0,0,0,1084,547,...,2440,0,0,0,572,0,17008,0,0,D
1,201807,TRAIN_000001,2565,2390,0,0,0,0,4090,2553,...,2677,0,2830,0,2736,0,0,0,0,E
2,201807,TRAIN_000002,5312,5113,21531,6795,0,0,5006,8778,...,9118,0,8870,0,4429,0,43351,0,0,C
3,201807,TRAIN_000003,730,5025,26284,0,0,0,487,5607,...,884,0,0,0,5097,0,30697,0,0,D
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,21,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,201812,TEST_99995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
2999996,201812,TEST_99996,191,0,0,0,0,0,260,0,...,137,0,0,0,0,0,0,0,0,None
2999997,201812,TEST_99997,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
2999998,201812,TEST_99998,17916,2786,0,0,0,0,16911,2081,...,18108,240,0,0,1830,0,0,0,0,None


### 결측치 찾기

In [4]:
# 전체 행 개수 계산
total_rows = len(df)

# 각 컬럼의 결측치 개수 계산
missing_count = df.isna().sum()

# 결측치 비율 계산 (전체 행 대비 %)
missing_pct = (missing_count / total_rows * 100).round(2)

# 결측치 정보 데이터프레임으로 정리
missing_info = pd.DataFrame({
    'missing_count': missing_count,
    'missing_pct': missing_pct
})

# 결측치가 있는 컬럼만 출력
print("\n결측치 정보 (개수 및 전체 대비 비율 %):")
print(missing_info[missing_info['missing_count'] > 0])


결측치 정보 (개수 및 전체 대비 비율 %):
          missing_count  missing_pct
연체일자_B0M        2992941        99.76
Segment          600000        20.00


### 값이 하나 뿐인 컬럼과 값이 모두 0인 컬럼도 제외

In [5]:
# 전부 0인 컬럼 찾기
zero_cols = [c for c in df.columns if df[c].eq(0).all()]

print("모두 0인 컬럼 (제거 대상):")
print(zero_cols)

# 전부 0인 컬럼 제거
df.drop(columns=zero_cols, inplace = True)

# 4) 결과 확인
print("최종 DataFrame shape:", df.shape)
print("남은 컬럼:", df.columns.tolist())

모두 0인 컬럼 (제거 대상):
['카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M', '연체잔액_CA_해외_B0M']
최종 DataFrame shape: (3000000, 75)
남은 컬럼: ['기준년월', 'ID', '잔액_일시불_B0M', '잔액_할부_B0M', '잔액_현금서비스_B0M', '잔액_리볼빙일시불이월_B0M', '잔액_리볼빙CA이월_B0M', '잔액_카드론_B0M', '월중평잔_일시불_B0M', '월중평잔_할부_B0M', '월중평잔_CA_B0M', '연체일자_B0M', '연체잔액_B0M', '연체잔액_일시불_B0M', '연체잔액_할부_B0M', '연체잔액_현금서비스_B0M', '연체잔액_카드론_B0M', '연체잔액_대환론_B0M', '잔액_현금서비스_B1M', '잔액_현금서비스_B2M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', '잔액_할부_B1M', '잔액_할부_B2M', '잔액_일시불_B1M', '잔액_일시불_B2M', '연체일수_B1M', '연체일수_B2M', '연체원금_B1M', '연체원금_B2M', '연체일수_최근', '연체원금_최근', '최종연체회차', '매각잔액_B1M', 'RV_평균잔액_R12M', 'RV_최대잔액_R12M', 'RV_평균잔액_R6M', 'RV_최대잔액_R6M', 'RV_평균잔액_R3M', 'RV_최대잔액_R3M', '잔액_할부_유이자_B0M', '잔액_할부_무이자_B0M', '잔액_할부_해외_B0M', '연체잔액_RV일시불_B0M', '연체잔액_CA_B0M', '월중평잔', '월중평잔_일시불', '월중평잔_RV일시불', '월중평잔_할부', '월중평잔_CA', '월중평잔_카드론', '평잔_3M', '평잔_일시불_3M', '평잔_일시불_해외_3M', '평잔_RV일시불_3M

In [6]:
# 모든 값이 동일한(상수) 컬럼 찾기
const_cols = [c for c in df.columns if df[c].nunique(dropna=False) == 1]

# 제거 대상 컬럼 목록 출력
print("모두 같은 값인 컬럼 (제거 대상):")
print(const_cols)

# 상수 컬럼 제거
df.drop(columns=const_cols, inplace=True)

# 결과 확인
print("최종 DataFrame shape:", df.shape)
print("남은 컬럼:", df.columns.tolist())

모두 같은 값인 컬럼 (제거 대상):
[]
최종 DataFrame shape: (3000000, 75)
남은 컬럼: ['기준년월', 'ID', '잔액_일시불_B0M', '잔액_할부_B0M', '잔액_현금서비스_B0M', '잔액_리볼빙일시불이월_B0M', '잔액_리볼빙CA이월_B0M', '잔액_카드론_B0M', '월중평잔_일시불_B0M', '월중평잔_할부_B0M', '월중평잔_CA_B0M', '연체일자_B0M', '연체잔액_B0M', '연체잔액_일시불_B0M', '연체잔액_할부_B0M', '연체잔액_현금서비스_B0M', '연체잔액_카드론_B0M', '연체잔액_대환론_B0M', '잔액_현금서비스_B1M', '잔액_현금서비스_B2M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', '잔액_할부_B1M', '잔액_할부_B2M', '잔액_일시불_B1M', '잔액_일시불_B2M', '연체일수_B1M', '연체일수_B2M', '연체원금_B1M', '연체원금_B2M', '연체일수_최근', '연체원금_최근', '최종연체회차', '매각잔액_B1M', 'RV_평균잔액_R12M', 'RV_최대잔액_R12M', 'RV_평균잔액_R6M', 'RV_최대잔액_R6M', 'RV_평균잔액_R3M', 'RV_최대잔액_R3M', '잔액_할부_유이자_B0M', '잔액_할부_무이자_B0M', '잔액_할부_해외_B0M', '연체잔액_RV일시불_B0M', '연체잔액_CA_B0M', '월중평잔', '월중평잔_일시불', '월중평잔_RV일시불', '월중평잔_할부', '월중평잔_CA', '월중평잔_카드론', '평잔_3M', '평잔_일시불_3M', '평잔_일시불_해외_3M', '평잔_RV일시불_3M', '평잔_RV일시불_해외_3M', '평잔_할부_3M', '평잔_할부_해외_3M', '평잔_CA_3M', '평잔_CA_해외_3M', '평잔_카드론_3M', '평잔_6M', '평잔_일시불_6M', '평잔_일시불_해외_6M', '평잔_RV일시

In [7]:
# 전체 컬럼 리스트 생성
cols = df.columns.tolist()

# 최빈값 비율이 99.9% 초과하는 컬럼 수집
low_variance_cols = []
for col in cols:
    # 계산: 컬럼별 최빈값 비율 확인
    top_ratio = df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_ratio > 0.999:
        low_variance_cols.append(col)

# 제거 대상 컬럼 목록 출력
print("제거 대상 컬럼:", low_variance_cols)

# 제거 대상 컬럼 삭제
df.drop(columns=low_variance_cols, inplace=True)

# 결과 확인
print("최종 컬럼 리스트:", df.columns.tolist())

제거 대상 컬럼: ['잔액_리볼빙CA이월_B0M', '연체잔액_현금서비스_B0M', '연체잔액_카드론_B0M', '연체잔액_대환론_B0M', '매각잔액_B1M', '잔액_할부_해외_B0M', '연체잔액_RV일시불_B0M', '연체잔액_CA_B0M', '평잔_할부_해외_3M', '평잔_CA_해외_3M', '평잔_CA_해외_6M']
최종 컬럼 리스트: ['기준년월', 'ID', '잔액_일시불_B0M', '잔액_할부_B0M', '잔액_현금서비스_B0M', '잔액_리볼빙일시불이월_B0M', '잔액_카드론_B0M', '월중평잔_일시불_B0M', '월중평잔_할부_B0M', '월중평잔_CA_B0M', '연체일자_B0M', '연체잔액_B0M', '연체잔액_일시불_B0M', '연체잔액_할부_B0M', '잔액_현금서비스_B1M', '잔액_현금서비스_B2M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', '잔액_할부_B1M', '잔액_할부_B2M', '잔액_일시불_B1M', '잔액_일시불_B2M', '연체일수_B1M', '연체일수_B2M', '연체원금_B1M', '연체원금_B2M', '연체일수_최근', '연체원금_최근', '최종연체회차', 'RV_평균잔액_R12M', 'RV_최대잔액_R12M', 'RV_평균잔액_R6M', 'RV_최대잔액_R6M', 'RV_평균잔액_R3M', 'RV_최대잔액_R3M', '잔액_할부_유이자_B0M', '잔액_할부_무이자_B0M', '월중평잔', '월중평잔_일시불', '월중평잔_RV일시불', '월중평잔_할부', '월중평잔_CA', '월중평잔_카드론', '평잔_3M', '평잔_일시불_3M', '평잔_일시불_해외_3M', '평잔_RV일시불_3M', '평잔_RV일시불_해외_3M', '평잔_할부_3M', '평잔_CA_3M', '평잔_카드론_3M', '평잔_6M', '평잔_일시불_6M', '평잔_일시불_해외_6M', '평잔_RV일시불_6M', '평잔_RV일시불_해외_6M', '평잔

### 비수치형 또는 혼합형 컬럼 찾아 숫자로 변환하기

In [8]:
# 각 컬럼별 분포(value_counts) 출력
c1 = list (df.columns)
for col in c1:
    print(f"▶ [{col}] 분포")
    print(df[col].value_counts(dropna=False))
    print()  # 빈 줄로 구분

▶ [기준년월] 분포
기준년월
201807    500000
201808    500000
201809    500000
201810    500000
201811    500000
201812    500000
Name: count, dtype: int64

▶ [ID] 분포
ID
TRAIN_000000    6
TRAIN_333343    6
TRAIN_333341    6
TRAIN_333340    6
TRAIN_333339    6
               ..
TRAIN_166662    6
TRAIN_166661    6
TRAIN_166660    6
TRAIN_166659    6
TEST_99999      6
Name: count, Length: 500000, dtype: int64

▶ [잔액_일시불_B0M] 분포
잔액_일시불_B0M
0        858154
7          1546
8          1544
9          1363
10         1211
          ...  
73352         1
39756         1
87865         1
39660         1
56513         1
Name: count, Length: 49951, dtype: int64

▶ [잔액_할부_B0M] 분포
잔액_할부_B0M
0        2038700
1601         344
1809         343
1714         343
1712         340
          ...   
37273          1
33015          1
35442          1
23363          1
27201          1
Name: count, Length: 28506, dtype: int64

▶ [잔액_현금서비스_B0M] 분포
잔액_현금서비스_B0M
0        2852256
8166          29
7372          25
6606         

In [9]:
# 검사할 컬럼 리스트
c1 = list (df.columns)
cols_to_check = c1

# 비수치형 컬럼 저장용 리스트
non_numeric_cols = []

# 각 컬럼에 대해 수치형 변환 시도 후 NaN 비율로 판단
for col in cols_to_check:
    # 결측을 빈 문자열로, 나머지를 문자열로 변환
    s = df[col].fillna('').astype(str)
    # 숫자로 변환 (불가능한 항목은 NaN)
    num = pd.to_numeric(s, errors='coerce')
    # 변환 불가 비율 계산
    na_ratio = num.isna().mean()
    # 일정 비율 이상이면 비수치형으로 간주 (여기선 100% 미수치형도 포함)
    if na_ratio > 0:
        non_numeric_cols.append(col)

# 결과 출력
print("비수치형(또는 혼합형) 컬럼:", non_numeric_cols)

비수치형(또는 혼합형) 컬럼: ['ID', '연체일자_B0M', 'Segment']


In [10]:
# 변환 전에 원본 분포 확인 (NaN 포함)
print("원본 연체일자_B0M 분포 (최다값 상위 10개):")
print(channel_df['연체일자_B0M']
      .value_counts(dropna=False)
      .head(10))

# 8자리 숫자만 뽑아서 datetime 으로 변환
digits = channel_df['연체일자_B0M'].astype(str).str.extract(r'(\d{8})')[0]
dates = pd.to_datetime(digits, format='%Y%m%d', errors='coerce')

# 변환 후 분포 확인
print("\n변환된 날짜 분포 (NaT 포함, 최다값 상위 10개):")
print(dates.value_counts(dropna=False).head(10))

# 실제 날짜만 보고 싶다면 dropna 처리
print("\n실제 날짜만 (NaT 제외, 최다값 상위 10개):")
print(dates.dropna().value_counts().head(10))

# 변환된 결과를 df 에 저장하고 싶다면
df['연체일자_B0M'] = dates

원본 연체일자_B0M 분포 (최다값 상위 10개):
연체일자_B0M
NaN           2992941
20180521.0        105
20180517.0        104
20180520.0         98
20180523.0         97
20180522.0         91
20180518.0         86
20180519.0         84
20180524.0         83
20180515.0         82
Name: count, dtype: int64

변환된 날짜 분포 (NaT 포함, 최다값 상위 10개):
0
NaT           2992941
2018-05-21        105
2018-05-17        104
2018-05-20         98
2018-05-23         97
2018-05-22         91
2018-05-18         86
2018-05-19         84
2018-05-24         83
2018-05-15         82
Name: count, dtype: int64

실제 날짜만 (NaT 제외, 최다값 상위 10개):
0
2018-05-21    105
2018-05-17    104
2018-05-20     98
2018-05-23     97
2018-05-22     91
2018-05-18     86
2018-05-19     84
2018-05-24     83
2018-05-15     82
2018-05-05     78
Name: count, dtype: int64


In [12]:
# drop 이후, 저장 직전의 컬럼과 0/NaN 컬럼 재확인
zero_cols = [c for c in df.columns if df[c].eq(0).all()]
na_cols   = [c for c in df.columns if df[c].isna().any()]

print("저장 직전 0 전부 컬럼:", zero_cols)
print("저장 직전 NaN 있는 컬럼:", na_cols)
print("저장 직전 전체 컬럼 수:", len(df.columns))

저장 직전 0 전부 컬럼: []
저장 직전 NaN 있는 컬럼: ['연체일자_B0M', 'Segment']
저장 직전 전체 컬럼 수: 64


In [13]:
# csv 파일로 저장
df.to_csv('잔액정보_전처리.csv',  index=False)

In [16]:
# 저장된 파일 불러와서 확인하기
df2 = pd.read_csv('잔액정보_전처리.csv')
zero_after = [c for c in df2.columns if df2[c].eq(0).all()]
na_after   = [c for c in df2.columns if df2[c].isna().any()]

print("로드 후 0 전부 컬럼:", zero_after)
print("로드 후 NaN 있는 컬럼:", na_after)
print("로드 후 전체 컬럼 수:", len(df2.columns))

로드 후 0 전부 컬럼: []
로드 후 NaN 있는 컬럼: ['연체일자_B0M', 'Segment']
로드 후 전체 컬럼 수: 64
